In [ ]:
import pymysql
import json
from datetime import date, timedelta
from decimal import Decimal
import pandas as pd



# con = create_engine("mysql+pymysql://root:@localhost:3306/prueba8?utf8mb4")
con = pymysql.connect(
    host= 'localhost',
    port= 3306,
    user= 'root',
    password= '',
    database= 'prueba7',
    charset= 'utf8mb4',
    cursorclass= pymysql.cursors.DictCursor
)
cursor = con.cursor()
print( con )
    

: 

In [ ]:
query = ( '''
with t1 as (
select
    id_finanzas_egreso as id,
    concat( fecha, ' ', hora ) as fecha_hora,
    'e' as tipo,
    cast(fecha as char) as fecha,
    cast(hora as char) as hora,
    cast(cant as double) as cant,
    cast(costo_tot as double) as costo_tot
from finanzas_egreso
union all
select
    id_finanzas_ingreso as id,
    concat( fecha, ' ', hora ) as fecha_hora,
    'i' as tipo,
    cast(fecha as char) as fecha,
    cast(hora as char) as hora,
    cast(cant as double) as cant,
    cast(costo_tot as double) as costo_tot
from finanzas_ingreso)
select * from t1 order by fecha_hora asc, id asc;
''')
cursor.execute(query)
data = cursor.fetchall()

datajson = json.dumps( data  )
dt = pd.DataFrame( data )
# cursor.close()
# con.close()


In [2]:
query = '''
with t1 as (
select
    id_operacion_egreso as id,
    fecha as fecha_hora,
    'e' as tipo,
    cast(cast(fecha as date) as char) as fecha,
    cast(cast(fecha as time) as char) as hora,
    cast(cant as double) as cant,
    cast(costo_tot as double) as costo_tot,
    cast(costo_uni as double) as costo_uni
from operacion_egreso
union all
select
    id_operacion_ingreso as id,
    fecha as fecha_hora,
    'i' as tipo,
    cast(cast(fecha as date) as char) as fecha,
    cast(cast(fecha as time) as char) as hora,
    cast(cant as double) as cant,
    cast(costo_tot as double) as costo_tot,
    cast(costo_uni as double) as costo_uni
from operacion_ingreso)
select * from t1 order by fecha asc, id asc;
'''
cursor.execute( query )
data = cursor.fetchall()
dt = pd.DataFrame( data )

In [3]:
def clasificar():
    
    for i, row in dt.iterrows():
        
        dt.loc[i, 'inversion'] = 0
        dt.loc[i, 'entrada'] = 0
        dt.loc[i, 'recuperacion'] = 0
        dt.loc[i, 'salida'] = 0
        
        if( row['tipo'] == 'e' ):
            dt.loc[i, 'inversion'] = row['costo_tot']
            dt.loc[i, 'entrada'] = row['cant']
        
        elif( row['tipo'] == 'i' ):
            dt.loc[i, 'recuperacion'] = row['costo_tot']
            dt.loc[i, 'salida'] = row['cant']

clasificar()
dt2 = dt[['id', 'fecha_hora', 'tipo', 'inversion', 'recuperacion', 'entrada', 'salida']].copy()
dt2['1'] = '|'

In [4]:
def calcularExistencia():
    
    entrada, salida = 0, 0
    for i, row in dt2.iterrows():
        
        entrada += row['entrada']
        salida += row['salida']
        
        dt2.loc[i, 'entrada_suma'] = entrada
        dt2.loc[i, 'salida_suma'] = salida
        dt2.loc[i, 'existencia'] = entrada - salida

calcularExistencia()
dt2 = dt2[['id', 'fecha_hora', 'tipo', 'entrada', 'inversion', 'salida', 'recuperacion', '1', 'existencia']]
dt2['2'] = '|'

In [5]:
def calcularSumaEgreso():
    
    inversion_suma, recuperacion_suma, salida_suma = 0, 0 ,0
    esExistencia0 = False
    
    for i, row in dt2.iterrows():
        
        if( esExistencia0 ):
            inversion_suma = row['inversion']
            recuperacion_suma = row['recuperacion']
            salida_suma = row['salida']
        
        else:
            inversion_suma += row['inversion']
            recuperacion_suma += row['recuperacion']
            salida_suma += row['salida']
            
        dt2.loc[i, 'inversion_suma'] = inversion_suma
        dt2.loc[i, 'recuperacion_suma'] = recuperacion_suma
        dt2.loc[i, 'salida_suma'] = salida_suma
        
        esExistencia0 = row['existencia'] == 0
        
        if( ( row['tipo'] == 'e' ) and ( row['existencia'] == 0 ) ):
            dt2.loc[i, 'costo_uni_sin_stock'] = inversion_suma / salida_suma

calcularSumaEgreso()
dt2['3'] = '|'
dt3 = pd.concat( [dt2, dt[['costo_uni']]], axis=1)

In [33]:
dt3

,id,fecha_hora,tipo,inversion,recuperacion,entrada,salida,1,existencia,2,inversion_suma,recuperacion_suma,salida_suma,costo_uni_sin_stock,3,costo_uni
0,6,2023-01-07 09:52:02,i,0.00,0.00,0.0,2.0,|,-2.0,|,0.00,0.00,2.0,NaN,|,0.00
1,15,2023-01-17 22:06:08,i,0.00,0.00,0.0,3.0,|,-5.0,|,0.00,0.00,5.0,NaN,|,0.00
2,7,2023-01-30 02:51:54,e,80.00,0.00,5.0,0.0,|,0.0,|,80.00,0.00,5.0,16.00,|,16.00
3,16,2023-03-17 19:59:15,i,0.00,0.00,0.0,3.0,|,-3.0,|,0.00,0.00,3.0,NaN,|,0.00
4,8,2023-03-23 15:12:20,i,0.00,0.00,0.0,4.0,|,-7.0,|,0.00,0.00,7.0,NaN,|,0.00
5,9,2023-03-31 14:02:33,e,70.00,0.00,5.0,0.0,|,-2.0,|,70.00,0.00,7.0,NaN,|,14.00
6,1,2023-04-10 02:37:27,e,16.66,0.00,2.0,0.0,|,0.0,|,86.66,0.00,7.0,12.38,|,8.33
7,10,2023-04-15 05:10:56,e,49.98,0.00,6.0,0.0,|,6.0,|,49.98,0.00,0.0,NaN,|,8.33
8,17,2023-04-22 14:37:50,i,0.00,24.99,0.0,3.0,|,3.0,|,49.98,24.99,3.0,NaN,|,8.33
9,19,2023-04-22 16:37:50,i,0.00,0.00,0.0,2.0,|,1.0,|,49.98,24.99,5.0,NaN,|,0.00


In [6]:
def asignarCostoUni():
    
    for i, row in dt3.iterrows():
        
        fecha_actual = row['fecha_hora']
        costo_uni_pro = dt3.loc[
            ( dt3['existencia'] == 0 ) &
            ( dt3['tipo'] == 'e' ) &
            ( dt3['fecha_hora'] >= fecha_actual )
        ]['costo_uni_sin_stock'].head(1)
        
        if( row['tipo'] == 'e' ):
            dt3.loc[i, 'costo_uni_pro'] = row['costo_uni']
        
        if( row['tipo'] == 'i'):
            if( row['existencia'] >= 0 and row['recuperacion'] != 0 ):
                dt3.loc[i, 'costo_uni_pro'] = row['costo_uni']
            
            elif( row['existencia'] < 0 ):
                dt3.loc[i, 'costo_uni_pro'] = costo_uni_pro.iloc[0] if not costo_uni_pro.empty else None
                
            else:
                dt3.loc[i, 'costo_uni_pro'] = None

asignarCostoUni()
dt3

,id,fecha_hora,tipo,entrada,inversion,salida,recuperacion,1,existencia,2,inversion_suma,recuperacion_suma,salida_suma,costo_uni_sin_stock,3,costo_uni,costo_uni_pro
0,6,2023-01-07 09:52:02,i,0.0,0.00,2.0,0.00,|,-2.0,|,0.00,0.00,2.0,NaN,|,0.00,16.00
1,15,2023-01-17 22:06:08,i,0.0,0.00,3.0,0.00,|,-5.0,|,0.00,0.00,5.0,NaN,|,0.00,16.00
2,7,2023-01-30 02:51:54,e,5.0,80.00,0.0,0.00,|,0.0,|,80.00,0.00,5.0,16.00,|,16.00,16.00
3,16,2023-03-17 19:59:15,i,0.0,0.00,3.0,0.00,|,-3.0,|,0.00,0.00,3.0,NaN,|,0.00,12.38
4,8,2023-03-23 15:12:20,i,0.0,0.00,4.0,0.00,|,-7.0,|,0.00,0.00,7.0,NaN,|,0.00,12.38
5,9,2023-03-31 14:02:33,e,5.0,70.00,0.0,0.00,|,-2.0,|,70.00,0.00,7.0,NaN,|,14.00,14.00
6,1,2023-04-10 02:37:27,e,2.0,16.66,0.0,0.00,|,0.0,|,86.66,0.00,7.0,12.38,|,8.33,8.33
7,10,2023-04-15 05:10:56,e,6.0,49.98,0.0,0.00,|,6.0,|,49.98,0.00,0.0,NaN,|,8.33,8.33
8,17,2023-04-22 14:37:50,i,0.0,0.00,3.0,24.99,|,3.0,|,49.98,24.99,3.0,NaN,|,8.33,8.33
9,19,2023-04-22 16:37:50,i,0.0,0.00,2.0,0.00,|,1.0,|,49.98,24.99,5.0,NaN,|,0.00,NaN
